## Preprocessing

In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
X = application_df.drop(columns=['EIN', 'NAME'])

In [14]:
# Determine the number of unique values in each column.
X.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [15]:
# Step 1: Define the threshold for replacing less frequent categories with "Other"
threshold = 100  # Replace categories with counts less than or equal to this threshold

# Step 2: Create a list of application types that should be replaced
application_counts = application_df['APPLICATION_TYPE'].value_counts()
replace_application = application_counts[application_counts <= threshold].index

# Step 3: Replace the identified application types with 'Other'
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(replace_application, 'Other')

# Step 4: Verify the transformation
print(application_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64


In [22]:
# Step 1: Define the threshold for replacing less frequent categories with "Other"
classification_threshold = 100  # Replace categories with counts less than or equal to this threshold

# Step 2: Analyze CLASSIFICATION value counts to identify categories to replace
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Step 3: Generate a list of classifications that will be replaced with "Other"
classifications_to_replace = classification_counts[classification_counts <= classification_threshold].index

# Step 4: Replace the identified classifications in the dataframe
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other")

# Step 5: Confirm that the replacement process was successful
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64


In [32]:
# Convert remaining categorical data to numeric with `pd.get_dummies`
X = pd.get_dummies(X, columns=['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 
                               'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

print(X.head())


   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0     5000              1                   False                  True   
1   108590              1                   False                 False   
2     5000              0                   False                 False   
3     6692              1                   False                 False   
4   142590              1                   False                 False   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                 False                False                False   
1                 False                 True                False   
2                 False                False                False   
3                 False                 True                False   
4                 False                 True                False   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                False                False                Fa

In [33]:
# Split our preprocessed data into our features and target arrays
y = X['IS_SUCCESSFUL']  # Target variable (dependent variable)
X = X.drop(columns=['IS_SUCCESSFUL'])  # Features (independent variables)

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split by printing the shapes of the datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (27439, 51)
X_test shape: (6860, 51)
y_train shape: (27439,)
y_test shape: (6860,)


In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]  

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=96, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=39, activation='relu'))

# Output layer
# Binary classification (1 or 0), so we use a single unit with sigmoid activation
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Check the structure of the model
nn.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 96)             │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 39)             │         3,783 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            40 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,815 (34.43 KB)

 Trainable params: 8,815 (34.43 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
# Train the model
history = nn.fit(X_train_scaled, y_train, 
                 epochs=100,  # Number of epochs
                 batch_size=32,  # Number of samples per batch
                 validation_data=(X_test_scaled, y_test))  # Evaluate on the test data after each epoch

# Print the training process history
print(history.history)


Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6965 - loss: 0.5966 - val_accuracy: 0.7229 - val_loss: 0.5605
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - accuracy: 0.7291 - loss: 0.5530 - val_accuracy: 0.7235 - val_loss: 0.5575
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7315 - loss: 0.5487 - val_accuracy: 0.7195 - val_loss: 0.5565
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7348 - loss: 0.5446 - val_accuracy: 0.7278 - val_loss: 0.5575
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 954us/step - accuracy: 0.7307 - loss: 0.5471 - val_accuracy: 0.7303 - val_loss: 0.5546
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7344 - loss: 0.5435 - val_accuracy: 0.7262 - val_loss: 0.5536
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7296 - loss: 0.5463 - val_accuracy: 0.7241 - val_loss: 0.5572
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7317 - loss: 0.5450 - val_

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 617us/step - accuracy: 0.7270 - loss: 0.5594
Loss: 0.559380829334259, Accuracy: 0.7269679307937622


In [41]:
# Export our model to HDF5 file
nn.save("trained_model.h5")

# Confirm the model was saved
print("Model saved to trained_model.h5")


Model saved to trained_model.h5
